## Installations

In [1]:
!pip install transformers

In [2]:
!pip install pytorch-lightning

## Repo + Imports

In [3]:
!rm -r '/content/NLP-2022L'
!git clone 'https://github.com/Adam-Kowalczyk/NLP-2022L.git'

Cloning into 'NLP-2022L'...
remote: Enumerating objects: 106, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 106 (delta 26), reused 55 (delta 15), pack-reused 34
Receiving objects: 100% (106/106), 1.04 MiB | 27.25 MiB/s, done.
Resolving deltas: 100% (32/32), done.


In [4]:
import sys

sys.path.append('/content/NLP-2022L/projects/team_4/src')

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from spacy.lang.pl import Polish

import string 
import re

import seaborn as sns
from collections import  Counter
from matplotlib.pyplot import figure

import torch

## Data Loading

In [6]:
!wget "http://2019.poleval.pl/task6/task_6-2.zip"

--2022-05-11 22:25:02--  http://2019.poleval.pl/task6/task_6-2.zip
Resolving 2019.poleval.pl (2019.poleval.pl)... 213.135.36.94
Connecting to 2019.poleval.pl (2019.poleval.pl)|213.135.36.94|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 340096 (332K) [application/zip]
Saving to: ‘task_6-2.zip.2’

task_6-2.zip.2      100%[===================>] 332.12K   535KB/s    in 0.6s    

2022-05-11 22:25:03 (535 KB/s) - ‘task_6-2.zip.2’ saved [340096/340096]



In [7]:
!unzip "/content/task_6-2.zip" -d "/content/data"

Archive:  /content/task_6-2.zip
replace /content/data/training_set_clean_only_text.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /content/data/training_set_clean_only_text.txt  
  inflating: /content/data/training_set_clean_only_tags.txt  


In [6]:
original_tags = pd.read_csv('/content/data/training_set_clean_only_tags.txt', header = None)
with open('/content/data/training_set_clean_only_text.txt', 'r') as textFile:
  lines = textFile.readlines()
original_text = pd.DataFrame(lines)

df = pd.concat([original_text,original_tags], axis=1)
df.columns =['Text', 'Tag']

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10041 entries, 0 to 10040
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    10041 non-null  object
 1   Tag     10041 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 157.0+ KB


## Data Preprocessing

In [8]:
from dataset.preprocessing import preprocess_text

In [9]:
nlp = Polish()

In [10]:
df['Tokens'] = df['Text'].map(lambda x: preprocess_text(x, nlp))

## Tfidf

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
RE_EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)

def identity_tokenizer(text):
    return [toc for toc in text if bool(RE_EMOJI.match(toc))]

tfidf_tokenizer = TfidfVectorizer(tokenizer=identity_tokenizer, lowercase=False) # token_pattern=r'[^\s]+'
tfidf_tokenizer.fit(df['Tokens'])

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


TfidfVectorizer(lowercase=False,
                tokenizer=<function identity_tokenizer at 0x7f8351ffb5f0>)

In [12]:
tfidf_tokenizer.get_feature_names_out()

array(['🆘', '🇦', '🇧', '🇬', '🇱', '🇵', '🌞', '🌧', '🌴', '🌹', '🌼', '🍀', '🍻',
       '🍼', '🍾', '🎁', '🎂', '🎈', '🎉', '🎤', '🎶', '🎷', '🎹', '🎺', '🏆', '🏴',
       '🏻', '🏻\u200d♀️', '🏻✌🏻', '🏻👍🏻',
       '🏼\\n\\n#ustawadegradacyjna\\n\\nhttps://t.co/a5lffolmwj',
       '🏼\u200d♀️', '🐇', '🐑', '🐕', '🐙', '🐝', '🐰', '🐱', '🐷', '🐻', '👇', '👊',
       '👋', '👌', '👍', '👎', '👏', '👐', '👪', '👰', '💁', '💐', '💓', '💔', '💕',
       '💛', '💝', '💞', '💪', '📚', '📺', '🔊', '🔜', '🔝', '🔥', '🔧', '🔪', '🔫',
       '🔯', '🕵', '🖒', '😀', '😁', '😁\\n', '😁😁😁😁😁.no', '😂',
       '😂😂😂\\n#woronicza17', '😃', '😄', '😅', '😆', '😇', '😉', '😊', '😋', '😍',
       '😎', '😐', '😑', '😒', '😓', '😔', '😖', '😘', '😛', '😜', '😝', '😞', '😡',
       '😢', '😣', '😥', '😧', '😨', '😩', '😬', '😭', '😮', '😯', '😱', '😳', '😷',
       '🙂', '🙃', '🙄', '🙆', '🙇', '🙈', '🙉', '🙊', '🙋', '🙌', '🙏', '🚜', '🤐',
       '🤔', '🤗', '🤣', '🤦', '🤨', '🤪', '🤭', '🤷', '🥀', '🥂', '🦀', '🧐',
       '\U000e0067\U000e0062\U000e0065\U000e006e\U000e0067\U000e007f'],
      dtype=object)

## Transformers

In [13]:
from transformers import AutoTokenizer, AutoModel

In [14]:
tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-large-cased")
model_bert = AutoModel.from_pretrained("allegro/herbert-large-cased")

Some weights of the model checkpoint at allegro/herbert-large-cased were not used when initializing BertModel: ['cls.sso.sso_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.sso.sso_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Dataset

In [15]:
from dataset.pytorch_dataset import HateSpeechDataset

In [16]:
torch.manual_seed(46)

hs_dataset = HateSpeechDataset(df, tokenizer, tfidf_tokenizer)
train_size = int(len(hs_dataset)*0.8)
val_size = int(len(hs_dataset)*0.1)
train_ds, val_ds, test_ds = torch.utils.data.random_split(hs_dataset, 
                                                  [train_size, val_size, len(hs_dataset) - train_size - val_size])

In [17]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_ds, batch_size=16, num_workers=2)

val_loader = DataLoader(val_ds, batch_size=16, num_workers=2)

test_loader = DataLoader(test_ds, batch_size=16, num_workers=2)

## Training

In [18]:
import pytorch_lightning as pl

from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint

from sklearn.utils.class_weight import compute_class_weight

from models.hate_speech_classifier import HateSpeechClassifier
from models.mlp_pytorch import MLP
from training.model_lightning import ModelLightining

In [19]:
model_classifier = MLP([139 + 1024, 3])

model = HateSpeechClassifier(model_bert, model_classifier, train_bert = False) # 'True' causes 'CUDA out of memory' in colab :/

class_weights = compute_class_weight(class_weight = 'balanced', classes = np.unique(df['Tag']), y= df['Tag']).astype(np.float32)
print(class_weights)

model_pl = ModelLightining(model, class_weights, lr = 1e-3)

[ 0.3642002 13.229249   5.59699  ]


In [20]:
early_stop_callback = EarlyStopping(monitor="val_loss", patience=3, verbose = True)
model_checkpoint_callback = ModelCheckpoint(monitor="val_loss", dirpath = 'MLP')
trainer = pl.Trainer(max_epochs=5, devices=1, accelerator="auto", 
                     callbacks=[early_stop_callback, model_checkpoint_callback])

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [21]:
trainer.fit(model_pl, train_dataloaders=train_loader, val_dataloaders=val_loader)

Missing logger folder: /content/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                 | Params
-------------------------------------------------------
0 | model         | HateSpeechClassifier | 355 M 
1 | train_metrics | MetricCollection     | 0     
2 | valid_metrics | MetricCollection     | 0     
3 | f1_per_class  | F1Score              | 0     
-------------------------------------------------------
3.5 K     Trainable params
355 M     Non-trainable params
355 M     Total params
1,420.372 Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]


 Val F1 per class: tensor([0.7981, 0.2000, 0.0000], device='cuda:0')


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Metric val_loss improved by 0.021 >= min_delta = 0.0. New best score: 0.867



 Val F1 per class: tensor([0.9410, 0.0000, 0.2906], device='cuda:0')


Validation: 0it [00:00, ?it/s]

Metric val_loss improved by 0.036 >= min_delta = 0.0. New best score: 0.831



 Val F1 per class: tensor([0.9396, 0.0000, 0.2896], device='cuda:0')


In [22]:
trainer.validate(ckpt_path = 'best', dataloaders = test_loader)

Restoring states from the checkpoint path at /content/MLP/epoch=4-step=2510.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at /content/MLP/epoch=4-step=2510.ckpt


Validation: 0it [00:00, ?it/s]


 Val F1 per class: tensor([0.9244, 0.0000, 0.3012], device='cuda:0')
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      val_accuracy          0.8577114343643188
        val_loss            0.8730936646461487
       val_macroF1           0.543006420135498
       val_microF1          0.8577114343643188
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'val_accuracy': 0.8577114343643188,
  'val_loss': 0.8730936646461487,
  'val_macroF1': 0.543006420135498,
  'val_microF1': 0.8577114343643188}]